# OTRO EJEMPLO

- Este no funcionó y es más simple

In [ ]:
class SimpleCoughNetWithCNN(torch.nn.Module):
    def __init__(self, scalar_input_size):
        super(SimpleCoughNetWithCNN, self).__init__()
        
        # Simple CNN for spectrogram (input: batch x 1 x 128 x 128)
        self.spec_conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # -> 16 x 128 x 128
            nn.ReLU(),
            nn.MaxPool2d(2),                             # -> 16 x 64 x 64
            nn.Conv2d(16, 32, kernel_size=3, padding=1), # -> 32 x 64 x 64
            nn.ReLU(),
            nn.MaxPool2d(2),                             # -> 32 x 32 x 32
        )
        
        # Simple CNN for chromagram (input: batch x 1 x 12 x 128)
        self.chroma_conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # -> 16 x 12 x 128
            nn.ReLU(),
            nn.MaxPool2d(2),                             # -> 16 x 6 x 64
        )
        
        # Calculate CNN output sizes based on input dimensions
        self.spec_flat_size = 32 * 32 * 32      # = 32,768
        self.chroma_flat_size = 16 * 6 * 64     # = 6,144
        
        # Fully connected layers
        combined_size = scalar_input_size + self.spec_flat_size + self.chroma_flat_size
        self.fc_layers = nn.Sequential(
            nn.Linear(combined_size, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, scalar_x, spectrogram, chromagram):
        # Add batch dimension if needed
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram.unsqueeze(1)  # batch x 1 x 128 x 128
        if len(chromagram.shape) == 3:
            chromagram = chromagram.unsqueeze(1)    # batch x 1 x 12 x 128
        
        # Process spectrograms and chromagrams
        spec_features = self.spec_conv(spectrogram)
        spec_features = spec_features.view(spec_features.size(0), -1)  # flatten to batch x 32,768
        
        chroma_features = self.chroma_conv(chromagram)
        chroma_features = chroma_features.view(chroma_features.size(0), -1)  # flatten to batch x 6,144
        
        # Combine all features
        combined = torch.cat([scalar_x, spec_features, chroma_features], dim=1)
        
        return self.fc_layers(combined)

# Test the model
def test_model(scalar_input_size):
    # Create test inputs
    batch_size = 2
    scalar_x = torch.randn(batch_size, scalar_input_size)
    spec_x = torch.randn(batch_size, 128, 128)
    chroma_x = torch.randn(batch_size, 12, 128)
    
    # Initialize model
    model = SimpleCoughNetWithCNN(scalar_input_size)
    
    # Forward pass
    outputs = model(scalar_x, spec_x, chroma_x)
    
    print(f"Model test with batch size {batch_size}:")
    print(f"Scalar input shape: {scalar_x.shape}")
    print(f"Spectrogram input shape: {spec_x.shape}")
    print(f"Chromagram input shape: {chroma_x.shape}")
    print(f"Output shape: {outputs.shape}")
    
    return model

In [ ]:
# Initialize model and training components
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2 = SimpleCoughNetWithCNN(scalar_input_size=len(hparams['features'])).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Initialize tensorboard summary writer
time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
writer = SummaryWriter(f'logs/{time_stamp}/')

# Add graph to tensorboard
sample = next(iter(train_loader))
scalar_features, spectrograms, chromagrams = sample[0][:1], sample[1][:1], sample[2][:1]
writer.add_graph(model, (scalar_features, spectrograms, chromagrams))

# Training loop
best_val_loss = float('inf')
best_model_state = None

print("Starting training...")
for epoch in range(hparams['epochs']):
    train_loss, train_acc = train(train_loader, model2, optimizer, epoch)
    val_loss, val_acc = evaluate(val_loader, model2, epoch)
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model2.state_dict().copy()
        print(f'New best model saved with validation loss: {val_loss:.4f}')
    
    print(f'Epoch {epoch+1}/{hparams["epochs"]}:')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

# Save the best model
torch.save({
    'epoch': epoch,
    'model_state_dict': best_model_state,
    'optimizer_state_dict': optimizer.state_dict(),
    'best_val_loss': best_val_loss,
}, f'checkpoints/best_model_simpler_{time_stamp}.pth')

# Close tensorboard
writer.close()

print("Training completed!")
print("To view training progress, run:")
print("tensorboard --logdir logs")